If using google drive please edit this line to connect to drive location

In [1]:
%load_ext autoreload
%autoreload 2


In [100]:
from google.colab import drive
import os, sys

# Mount the google colab
drive.mount("/content/drive/")
GOOGLE_DRIVE_PATH_POST_MYDRIVE = 'DeepLearning_GroupProject/'
GOOGLE_DRIVE_PATH = os.path.join('/content', 'drive', 'MyDrive', GOOGLE_DRIVE_PATH_POST_MYDRIVE)
!ls {GOOGLE_DRIVE_PATH}/models
sys.path.append(GOOGLE_DRIVE_PATH)

# GOOGLE_DRIVE_PATH = '.'


# relative paths
models_dir = 'models'
csv_path = 'Data/updated_data_with_lists.csv'
tensors_dir = 'Data/tensor_batch_notaugmented'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
best_model_jake_12_10.pth	 recipe_checkpoint_12-08_23-15__Trial_2-best_model.pth
best_model_jake.pth		 recipe_checkpoint_12-09_22-53__Trial_4-best_model.pth
best_model.pth			 recipe_checkpoint_12-11_01-33__Trial_2-best_model.pth
image_2_recipe_bk.py		 recipe_embeddings_12_10_emilys.pth
image_2_recipe.py		 recipe_embeddings_12_10_jake_changegrad.pth
image_embeddings_proj_em_11.pth  recipe_embeddings_12_10_jake_noimage.pth
image_encoder.py		 recipe_embeddings_12_9_emilys.pth
MMR_jakes.py			 recipe_embeddings_12_9.pth
MMR_old.py			 recipe_embeddings_proj_em_11.pth
MMR.py				 recipe_embeddings.pth
MMR.py_clamp			 recipe_encoder_draft.py
__pycache__			 recipe_encoder.py


In [101]:
import pandas as pd
from ast import literal_eval
import numpy as np
import torch

# Explicitly adding models to the search path
models_path = os.path.join(GOOGLE_DRIVE_PATH, models_dir)
if models_path not in sys.path:
    sys.path.insert(0, models_path)

from models import recipe_encoder
from models import MMR_jakes
csv_file = os.path.join(GOOGLE_DRIVE_PATH,csv_path)
df = pd.read_csv(csv_file, converters={"Cleaned_Ingredients": literal_eval, "Instructions": literal_eval})
print(len(df))
df.head()

13496


,Unnamed: 0,Title,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"[Pat chicken dry with paper towels, season all...",miso-butter-roast-chicken-acorn-squash-panzanella,"[1 (3½–4-lb.) whole chicken, 2¾ tsp. kosher sa..."
1,1,Crispy Salt and Pepper Potatoes,[Preheat oven to 400°F and line a rimmed bakin...,crispy-salt-and-pepper-potatoes-dan-kluger,"[2 large egg whites, 1 pound new potatoes (abo..."
2,2,Thanksgiving Mac and Cheese,[Place a rack in middle of oven; preheat to 40...,thanksgiving-mac-and-cheese-erick-williams,"[1 cup evaporated milk, 1 cup whole milk, 1 ts..."
3,3,Italian Sausage and Bread Stuffing,[Preheat oven to 350°F with rack in middle. Ge...,italian-sausage-and-bread-stuffing-240559,"[1 (¾- to 1-pound) round Italian loaf, cut int..."
4,4,Newton's Law,[Stir together brown sugar and hot water in a ...,newtons-law-apple-bourbon-cocktail,"[1 teaspoon dark brown sugar, 1 teaspoon hot w..."


Concatenate the batches of preprocessed images into 1 tensor

In [102]:
pt_files = os.listdir(os.path.join(GOOGLE_DRIVE_PATH,tensors_dir))
all_image_tensors = []
all_image_labels = []

# Load and combine all .pt files
file_name = "batch_"
# for pt_file in pt_files[:5]:
for num in range(0,5):
    pt_file = file_name + str(num) + ".pt"
    pt_filepath = os.path.join(GOOGLE_DRIVE_PATH,tensors_dir,pt_file)
    image_tensors, image_labels = torch.load(pt_filepath)
    all_image_tensors.append(image_tensors)
    all_image_labels.extend(image_labels)
    print(f"Loaded {pt_file}")

# Concatenate tensors
all_image_tensors = torch.cat(all_image_tensors)
print(f"Number of images: {all_image_tensors.size(0)}")
print(f"Number of labels: {len(all_image_labels)}")
assert all_image_tensors.size(0) == len(all_image_labels), "Mismatch between images and labels!"

<ipython-input-102-ae3aa98ed976>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  image_tensors, image_labels = torch.load(pt_filepath)


Loaded batch_0.pt
Loaded batch_1.pt
Loaded batch_2.pt
Loaded batch_3.pt
Loaded batch_4.pt
Number of images: 5000
Number of labels: 5000


Reorganize dataframe to be in the same order as the Image Tensors using the image_label as the matching key

In [103]:
##Reset order of dataframe to match the image labels orders
all_image_labels_cleaned = [label.split(".")[0] for label in all_image_labels] #remove the .png and .jgp
print(all_image_labels_cleaned[:10]) #print to see if at the end the df matches

filtered_df = df[df["Image_Name"].isin(all_image_labels_cleaned)] #filter the df to only have values from the images
print(filtered_df["Image_Name"][:10])
print(df.shape)
print(filtered_df.shape, len(all_image_labels_cleaned))

valid_labels = set(filtered_df['Image_Name'])

# Filter labels and tensors
filtered_labels_and_tensors = [
    (tensor, label) for tensor, label in zip(all_image_tensors, all_image_labels_cleaned) if label in valid_labels
]

# Unpack the filtered data
filtered_tensors, filtered_labels = zip(*filtered_labels_and_tensors)

# Convert back to tensors
filtered_tensors = torch.stack(filtered_tensors)
filtered_labels = list(filtered_labels)

# Verify alignment
print(f"Number of filtered tensors: {len(filtered_tensors)}")
print(f"Number of filtered labels: {len(filtered_labels)}")
print(f"Number of rows in filtered_df: {len(filtered_df)}")

# Finally reorganize the df to be in the same order as the image tensors
filtered_df = filtered_df.set_index("Image_Name").loc[filtered_labels].reset_index()
print(filtered_df["Image_Name"][:10])

['spanish-style-grilled-vegetables-with-breadcrumb-picada-238806', 'three-cheese-pizza-with-onion-sage-and-arugula-233543', 'speedy-skillet-ravioli-lasagna', 'summer-anchovy-salad-51108430', 'spice-roasted-cornish-hens-with-cucumber-yogurt-sauce-353418', 'stovetop-butterscotch-apples-and-cranberries', 'tarte-tatin-51196820', 'spanish-olive-and-cream-cheese-canapes-231160', 'spice-rubbed-sustainable-fish-sliders', 'spiced-pumpkin-layer-cake-240123']
2            thanksgiving-mac-and-cheese-erick-williams
5                  warm-comfort-tequila-chamomile-toddy
7                     turmeric-hot-toddy-claire-sprouse
9        spiced-lentil-and-caramelized-onion-baked-eggs
11          spiral-ham-in-the-slow-cooker-guarnaschelli
19    roasted-beets-with-crispy-sunchokes-and-pickle...
24    sloppy-joe-shirred-eggs-with-spinach-vivian-ho...
27                           spicy-coconut-pumpkin-soup
30                                trinidad-curry-powder
31                                  shrimp-

Reformat Ingredients, Recipes, and Image titles

Tokenize Recipes, Ingredients, and Image Titles

In [104]:
from transformers import AutoTokenizer
total_max = 128
# Initialize the tokenizer
tokenizer_recipes = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_nested_list(nested_list, max_length=128):
    """
    Tokenizes a nested list of strings (list of ingredients per recipe).
    Each inner list is tokenized into a list of token IDs.
    """
    tokenized_list = []
    for sublist in nested_list:
        # Join the inner list into a string
        # text = " ".join(sublist)
        text = str(sublist)
        # Tokenize the string
        tokens = tokenizer_recipes(
            text,
            max_length=max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        # Append tokenized input_ids to the result list
        tokenized_list.append(tokens["input_ids"].squeeze(0).tolist())
    return tokenized_list

filtered_df['Title_List'] = df['Title'].apply(lambda x: [x])
filtered_df['tokenized_ingredients'] = filtered_df['Cleaned_Ingredients'].apply(lambda x: tokenize_nested_list(x, total_max))
filtered_df['tokenized_instructions'] = filtered_df['Instructions'].apply(lambda x: tokenize_nested_list(x, total_max))
filtered_df['tokenized_titles'] = filtered_df['Title_List'].apply(lambda x: tokenize_nested_list(x, total_max))


print(len(filtered_df["tokenized_titles"][0]))
print(filtered_df["tokenized_ingredients"][0])
print(filtered_df["tokenized_instructions"][0])


1
[[101, 1017, 2312, 2417, 4330, 23582, 1006, 2055, 1015, 1015, 1013, 1016, 7038, 1007, 1010, 27674, 1010, 13916, 1010, 4284, 2098, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1018, 2312, 2887, 8288, 24759, 11390, 1006, 2055, 1015, 1015, 1013, 1018, 7038, 1007, 1010, 21920, 1010, 3013, 3091, 14244, 2046, 1017, 25609, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1018, 5396, 2665, 2030, 3756, 16950, 25955, 3490, 1006, 9544, 8231, 1016, 

In [105]:
def pad_or_truncate_nested(sublist, target_length, max_length, pad_token=0):
        """
            Pad or truncate the outer list of a nested list to match the target_length.
            Each inner list remains untouched.
        """
        # Pad with [pad_token] or truncate the outer list
        if len(sublist) < target_length:
            sublist.extend([[pad_token]* max_length] * (target_length - len(sublist)))
        else:
            sublist = sublist[:target_length]
        return sublist

max_length_ing = filtered_df['tokenized_ingredients'].apply(len).max()
max_length_inst = filtered_df['tokenized_instructions'].apply(len).max()//4
max_length_title = filtered_df['tokenized_titles'].apply(len).max()
filtered_df['tokenized_ingredients'] = filtered_df['tokenized_ingredients'].apply(
    lambda ing: pad_or_truncate_nested(ing, max_length_ing,total_max))

filtered_df['tokenized_instructions'] = filtered_df['tokenized_instructions'].apply(
    lambda inst: pad_or_truncate_nested(inst, max_length_inst, total_max))
# new_token_ing = [pad_or_truncate_nested(ing, max_length_title) for ing in tokenized_ingredients] #titles were all list length of 1
max_length_ing = filtered_df['tokenized_ingredients'].apply(len).max()
max_length_inst = filtered_df['tokenized_instructions'].apply(len).max()
max_length_title = filtered_df['tokenized_titles'].apply(len).max()
print(max_length_ing, max_length_inst, max_length_title)
print(len(filtered_df['tokenized_ingredients'][5]))
print(filtered_df['tokenized_ingredients'][100])
print(filtered_df.shape)

42 23 1
42
[[101, 1015, 1013, 1018, 2452, 9724, 3514, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1016, 3009, 24444, 1010, 20956, 1998, 4857, 2135, 15920, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1016, 2312, 18856, 21818, 2015, 20548, 1010, 20956, 1998, 24881, 1010, 4606, 1018, 18856, 21818, 2015, 2878, 20548, 1010, 20956, 102, 0, 0, 0, 0, 0, 0, 0,

Tokenize the Image Labels for the Image Encoder

In [106]:
from transformers import AutoTokenizer
from transformers import CLIPModel
model_name = "openai/clip-vit-base-patch16"
clip_model = CLIPModel.from_pretrained(model_name)
tokenizer_images = AutoTokenizer.from_pretrained(model_name)

tokenized_labels = tokenizer_images(
    filtered_labels,
    padding="max_length",
    truncation=True,
    max_length=tokenizer_images.model_max_length,
    return_tensors="pt"
)

In [26]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
%reload_ext autoreload


cuda:0


In [ ]:
import torch.nn as nn
class RecipeDecoder(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(RecipeDecoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, vocab_size)
        self.hidden_dim = hidden_dim

    def forward(self, input_tokens):
        embedded = self.embedding(input_tokens)
        outputs, hidden_state = self.rnn(embedded, hidden_state)
        predictions = self.fc_out(outputs)
        return predictions

In [12]:
def calc_recall_name(top, image_features, recipe_embeddings, image_indices):
  true_pos = 0
  false_neg = 0
  tmp_batches = image_features.shape[0]
  # image_features = torch.nn.functional.normalize(image_features, dim=1)
  # recipe_embeddings = torch.nn.functional.normalize(recipe_embeddings, dim=1)
  top_embedded = []
  for i in range(tmp_batches):
    cosine_similarity = torch.nn.CosineSimilarity(dim=1)
    # similarities = cosine_similarity(image_features[i, :].unsqueeze(0), recipe_embeddings)
    similarities = cosine_similarity(image_features[i,:], recipe_embeddings)
    #top = amount of top results to retrieve
    top_results = top
    top_k_values, top_k_indices = torch.topk(similarities, top_results, largest=True)
    top_k_indices = top_k_indices.tolist()
    # top_images = [filtered_df['Image_Name'].iloc[i] for i in top_k_indices]
    # top_embedded.append(recipe_embeddings[top_k_indices[0]].detach().cpu().numpy())
    # print(top_k_values, top_k_indices)
    # print("Correct Image label: ", image_labels[i])
    print(top_k_indices)
    # print(top_images)
    # image_labels_str = image_labels[i].split(".")[0]
    if image_indices[i] in top_k_indices:
      print("found")
      true_pos += 1
    else:
      false_neg += 1
    # if image_labels_str in [image for image in top_images]:
    #   print("found")
    #   true_pos += 1
    # else:
    #   false_neg += 1
  print("total found = ", true_pos)
  # plot_2D(top_embedded, image_features, multiplier=None, batch_size=None)
  return true_pos


In [91]:
def calc_recall_showimages(top, image_features, recipe_embeddings, image_indices, selected_images, image_label):
  true_pos = 0
  false_neg = 0
  tmp_batches = image_features.shape[0]
  image_features = torch.nn.functional.normalize(image_features, dim=1)
  recipe_embeddings = torch.nn.functional.normalize(recipe_embeddings, dim=1)
  top_embedded = []
  for i in range(tmp_batches):
    cosine_similarity = torch.nn.CosineSimilarity(dim=1)
    similarities = cosine_similarity(image_features[i,:], recipe_embeddings)
    top_results = top
    top_k_values, top_k_indices = torch.topk(similarities, top_results, largest=True)
    top_k_indices = top_k_indices.tolist()
    # print(len(image_label), top_k_indices)
    # print(i, selected_images)
    if i in selected_images:
      top_images = [image_label[num] for num in top_k_indices]
      print("----------------------------")
      print("Correct Image ",image_label[image_indices[i]])
      print("Retrieved Images ",top_images)
      print("----------------------------")
    if image_indices[i] in top_k_indices:
      true_pos += 1
      top_images = [image_label[num] for num in top_k_indices]
      print("----------------------------")
      print("Correct Image ",image_label[image_indices[i]])
      print("Retrieved Images ",top_images)
      print("----------------------------")
    else:
      false_neg += 1
  print("total found = ", true_pos)
  return true_pos

In [112]:
def calc_recall_id(top, image_features, recipe_embeddings, image_indices):
  true_pos = 0
  false_neg = 0
  tmp_batches = image_features.shape[0]
  image_features = torch.nn.functional.normalize(image_features, dim=1)
  recipe_embeddings = torch.nn.functional.normalize(recipe_embeddings, dim=1)
  top_embedded = []
  for i in range(tmp_batches):
    cosine_similarity = torch.nn.CosineSimilarity(dim=1)
    similarities = cosine_similarity(image_features[i,:], recipe_embeddings)
    top_results = top
    top_k_values, top_k_indices = torch.topk(similarities, top_results, largest=True)
    top_k_indices = top_k_indices.tolist()

    # print(top_k_indices)
    if image_indices[i] in top_k_indices:
      true_pos += 1
    else:
      false_neg += 1
  print("total found = ", true_pos)
  return true_pos

In [ ]:
###Function for printing images and labels to check that the ordering is correct

mean = [0.485, 0.456, 0.406]  # Mean for RGB channels
std = [0.229, 0.224, 0.225]   # Std for RGB channels
import matplotlib.pyplot as plt

# Function to unnormalize
def unnormalize(tensor, mean, std):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)  # Scale by std, then add mean
    return tensor

def print_out_pic(images, image_labels):
    for i in range(images.shape[0]):
        tensor = images[i]
        image_label = image_labels[i]
        tensor = tensor.cpu()
        tensor = unnormalize(tensor, mean, std)  # Reverse normalization
        tensor = tensor.permute(1, 2, 0).numpy()  # Rearrange to HWC format

        plt.imshow(tensor)
        plt.title(image_label)
        plt.axis('off')
        plt.show()

In [13]:
###Plot in tSNE to debug
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
def plot_2D(recipe_embeddings, image_embeddings, multiplier=None, batch_size=None):
  batches = image_embeddings.shape[0]
  try:
    recipe_embeddings = recipe_embeddings.detach().cpu().numpy()
  except:
    recipe_embeddings = recipe_embeddings
  # print(type(recipe_embeddings))
  image_embeddings = image_embeddings.detach().cpu().numpy()
  combined_embeddings = np.concatenate((recipe_embeddings, image_embeddings), axis=0)
  x_list = np.arange(512)
  # print(x_list.shape)
  # print(recipe_embeddings[0])
  embedding_size = image_embeddings.shape[1]
  for batch in range(batches):
    plt.figure(figsize=(8, 8))
    if multiplier is not None:
      rec_batch = (multiplier*batch_size) + batch
      plt.scatter(np.arange(embedding_size), recipe_embeddings[rec_batch,:], label=f"Recipe Embeddings {rec_batch}", alpha=0.7)
      plt.scatter(np.arange(embedding_size), image_embeddings[batch,:], label=f"Image Embeddings {rec_batch}", alpha=0.7)
    else:
      plt.scatter(np.arange(embedding_size), recipe_embeddings[batch], label=f"Recipe Embeddings {batch}", alpha=0.7)
      plt.scatter(np.arange(embedding_size), image_embeddings[batch], label=f"Image Embeddings {batch}", alpha=0.7)
    plt.legend()
    plt.title("2D Visualization of Image and Recipe Embeddings")
    plt.xlabel("Recipe Embedding")
    plt.ylabel("Image Embedding")
    plt.show()

def plot_tsne(recipe_embeddings_batches, image_embeddings_batches):
  batches = image_embeddings_batches.shape[0]
  # recipe_3d = recipe_embeddings_batches.unsqueeze(0)
  # image_3d = image_embeddings_batches.unsqueeze(0)
  combined_embeddings = torch.cat((recipe_embeddings_batches, image_embeddings_batches), dim=0)
  np_embeddings = combined_embeddings.detach().cpu().numpy()
  print(np_embeddings.shape)
  tsne = TSNE(n_components=2, random_state=42, perplexity=9)

  reduced_embeddings = tsne.fit_transform(np_embeddings)

  # Separate reduced embeddings for visualization
  # num_images = np_embeddings.shape[1]
  # reduced_image_embeddings = reduced_embeddings[:num_images]
  # reduced_recipe_embeddings = reduced_embeddings[num_images:]
  print(reduced_embeddings.shape)
  # Plot the embeddings
  plt.figure(figsize=(8, 8))
  for batch in range(batches):
    plt.scatter(reduced_embeddings[batch+batches,:], reduced_embeddings[batch,:], label=f"Embeddings {batch}", alpha=0.7)
  plt.legend()
  plt.title("t-SNE Visualization of Image and Recipe Embeddings")
  plt.xlabel("t-SNE Dim 1")
  plt.ylabel("t-SNE Dim 2")
  plt.show()

def plot_tsne_recipefull(recipe_embeddings_batches):
  batches = recipe_embeddings_batches.shape[0]
  # recipe_3d = recipe_embeddings_batches.unsqueeze(0)
  # image_3d = image_embeddings_batches.unsqueeze(0)
  # combined_embeddings = torch.cat((recipe_embeddings_batches, image_embeddings_batches), dim=0)
  np_embeddings = recipe_embeddings_batches.detach().cpu().numpy()
  print(np_embeddings.shape)
  tsne = TSNE(n_components=2, random_state=42, perplexity=30)

  reduced_embeddings = tsne.fit_transform(np_embeddings)

  # Separate reduced embeddings for visualization
  # num_images = np_embeddings.shape[1]
  # reduced_image_embeddings = reduced_embeddings[:num_images]
  # reduced_recipe_embeddings = reduced_embeddings[num_images:]
  print(reduced_embeddings.shape)
  # Plot the embeddings
  plt.figure(figsize=(8, 8))
  for batch in range(batches):
    if batch + 1 < batches:
      plt.scatter(reduced_embeddings[batch,:], reduced_embeddings[batch+1,:], label=f"Embeddings {batch}", alpha=0.7)
    else:
      break
  plt.legend()
  plt.title("t-SNE Visualization of Image and Recipe Embeddings")
  plt.xlabel("t-SNE Dim 1")
  plt.ylabel("t-SNE Dim 2")
  plt.show()

In [108]:
from torch.utils.data import Dataset, DataLoader, random_split
import torch
from models.image_2_recipe import Image2Recipe
from models.image_encoder import Image_Encoder
from models.recipe_encoder import RecipeEncoder
from models.MMR import MMR
from models.MMR import MMR_losses
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

class Data_Loading(Dataset):
    """
    Class to combine the Images, Labels, Recipes together to be used in combination when inputted into Model
    """
    def __init__(self, tokenized_ingredients, tokenized_instructions, tokenized_titles, image_tensors, image_labels):
        self.ingredients = torch.tensor(tokenized_ingredients, dtype=torch.int16)
        self.instructions = torch.tensor(tokenized_instructions, dtype=torch.int16)
        self.titles = torch.tensor(tokenized_titles, dtype=torch.int16)
        self.images = image_tensors
        self.image_labels = image_labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return {
            "ingredients": self.ingredients[idx],
            "instructions": self.instructions[idx],
            "titles": self.titles[idx],
            "images": self.images[idx],
            "image_labels": self.image_labels[idx]

        }


class Trainer(object):
    """
    Class designed to run ViT (train, evaluate, plot)
    """

    def __init__(self, **kwargs):
        """
        Initialize ViT
        """
        self.epochs = kwargs['epochs']
        self.optimizer_name = kwargs['optimizer']
        self.device = kwargs['device']
        self.batch_size = kwargs['batch_size']
        self.lr = kwargs['learning_rate']

        self.tokenized_ingredients = kwargs['ingredient_tokens']
        self.tokenized_instructions = kwargs['instruction_tokens']
        self.tokenized_title = kwargs['title_tokens']
        self.image_tensor = kwargs['image_tensors']
        self.image_labels = kwargs['image_labels']
        self.clip_model = kwargs['clip_model']
        self.vocab_size = kwargs['vocab_size']
        self.max_len = kwargs['max_len']
        self.instance_weight = kwargs['instance_weight']
        self.sem_weight = kwargs['sem_weight']
        self.itm_weight = kwargs['itm_weight']
        self.best_model_parameters = kwargs['best_model_parameters_path']
        self.initial_margin = kwargs['initial_margin']
        self.margin_step = kwargs['margin_step']
        self.max_margin = kwargs['max_margin']
        self.topk = kwargs['topk']
        self.fixed_margin = kwargs['fixed_margin']
        self.improvement_threshold = kwargs['improvement_threshold']
        self.patience = kwargs['patience']
        self.best_model_parameters = kwargs['best_model_parameters_path']
        mode = 'optuna'

        self.mmr_heads = kwargs['mmr_heads']
        self.ITEM_lyrs = kwargs['ITEM_lyrs']
        self.MTD_lyrs = self.ITEM_lyrs
        self.hidden_dim = kwargs['hidden_dim']
        # self.recipe_enc_hidden = kwargs['recipe_enc_hidden']
        self.projection_dim=kwargs['projection_dim']
        self.MMR_type = kwargs['MMR_type']
        self.loss_calcs = MMR_losses(instance_weight=self.instance_weight, sem_weight=self.sem_weight, itm_weight=self.itm_weight)
        num_classes = len(set(self.image_labels))


        self.image_encoder = Image_Encoder(self.device, self.clip_model, num_classes).to(self.device)
        self.recipe_encoder = RecipeEncoder(self.device, self.vocab_size, self.max_len).to(self.device)
        # self.mmr = MMR(hidden_dim=self.image_encoder.clip_model.config.projection_dim).to(self.device)
        self.mmr = MMR(hidden_dim=self.image_encoder.clip_model.config.projection_dim, num_heads=self.mmr_heads, ITEM_lyrs=self.ITEM_lyrs, MTD_lyrs=self.MTD_lyrs, projection_dim=self.projection_dim, MMR_type=self.MMR_type).to(self.device)

        # self.recipe_decoder = RecipeDecoder(512,512,self.vocab_size).to(self.device)
        # MMR varaibles: num_heads=num_heads, ITEM_lyrs=ITEM_lyrs, MTD_lyrs=MTD_lyrs, projection_dim=projection_dim
        self.model = Image2Recipe(self.image_encoder, self.recipe_encoder, self.mmr).to(self.device)

        for param in self.model.image_encoder.parameters():
          param.requires_grad = False

        ##DO we want to tune each of these learning rates for each model?
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        # self.optimizer2 = torch.optim.Adam(self.recipe_decoder.parameters(), lr=self.lr)
        # self.optimizer = torch.optim.AdamW([
        #     {"params": self.model.image_encoder.parent_model.parameters(), "lr": 1e-6},
        #     {"params": self.model.recipe_encoder.parameters(), "lr": 1e-5},
        #     {"params": self.model.image_encoder.fc1.parameters(), "lr": 1e-5},
        #     {"params": self.model.recipe_encoder.ll_e.parameters(), "lr": 1e-5},
        # ])


        #Combine Images, Recipes, Instructions in training and eval datasets
        self.data_total = Data_Loading(
            self.tokenized_ingredients,
            self.tokenized_instructions,
            self.tokenized_title,
            self.image_tensor,
            self.image_labels
        )
        training_perc = .9
        train_size = int(training_perc * len(self.data_total))
        eval_size = len(self.data_total) - train_size
        train_dataset, eval_dataset = random_split(self.data_total, [train_size, eval_size])
        self.dataloader = {}
        self.dataloader['train'] = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, drop_last=True)
        self.dataloader['eval'] = DataLoader(eval_dataset, batch_size=self.batch_size, shuffle=False, drop_last=True)

        #Lists to fill up during training and plotted later for learning curves
        self.train_loss_list = []
        self.eval_loss_list = []
        self.eval_acc_list = []
        self.eval_acc_list = []
        self.best_score = 0
        self.best_loss = 10000
        print("finished initializing")

        # self.CELoss = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
        # self.decoder_lambda = kwargs['decoder_lambda']

    def train(self):
        """
        Train ViT, image encoder, recipe encoder, MMR
        """
        # Set initial margin
        self.margin = self.initial_margin

        for epoch in range(self.epochs):
            if epoch == 15:
              for param in self.model.image_encoder.parameters():
                param.requires_grad = True
                print("unfreezing image encoder")
              self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)

            for phase in ['train', 'eval']:
                total_loss = 0
                total_accuracy = 0
                all_recipe_embeddings = []
                all_image_embeddings = []
                all_image_labels = []
                total_loss_actual = 0
                total_eval_loss = 0
                if phase == 'train':
                    self.model.train()
                else:
                    self.model.eval()
                for i, batch_data in enumerate(tqdm(self.dataloader[phase],position=0, leave=True)):
                    #Looping through batches of training data then eval data each epoch
                    #TODO: Add how the recipe, instructions, and titles will be tokenized
                    ingredients, instructions, titles, images, image_labels = (
                        batch_data['ingredients'].to(self.device),
                        batch_data['instructions'].to(self.device),
                        batch_data['titles'].to(self.device),
                        batch_data['images'].to(self.device),
                        batch_data['image_labels']
                    )

                    recipe_enc_src = [titles, ingredients, instructions]

                    self.optimizer.zero_grad()

                    if phase == 'train':
                        output = self.model(images, image_labels, recipe_enc_src)
                        ##Combine the Recipe Encoder Losses and Image Encoder Losses based on TFOOD
                        mmr_logits = output["mmr_logits"]
                        image_logits = output["image_logits"]
                        image_embeddings_proj = output["image_embeddings_proj"]
                        recipe_embeddings_proj = output["recipe_embeddings_proj"]
                        # image_embeddings = output["image_embeddings"]
                        # recipe_embeddings = output["recipe_embeddings"]
                        # plot_2D(recipe_embeddings_proj, image_embeddings_proj)
                        # #Train the Recipe Decoder so it can output a generative (Title, Ingredient, Instructions)
                        # title_logits = self.recipe_decoder(image_embeddings, titles)
                        # ingredients_logits = self.recipe_decoder(image_embeddings, ingredients)
                        # instructions_logits = self.recipe_decoder(image_embeddings, instructions)

                        # #Compute Reconstruction Loss
                        # title_loss = self.CELoss(title_logits.transpose(1, 2), titles)
                        # ingredients_loss = self.CELoss(ingredients_logits.transpose(1, 2), ingredients)
                        # instructions_loss = self.CELoss(instructions_logits.transpose(1, 2), instructions)
                        # decoder_loss = (title_loss + ingredients_loss + instructions_loss) *self.decoder_lambda

                        loss = self.loss_calcs.total_loss(image_logits, image_embeddings_proj,
                                                          recipe_embeddings_proj, mmr_logits, self.margin)

                        print(f'training loss for step: {loss.item()}')
                        # recall_score = calc_recall(self.topk, image_embeddings_proj, recipe_embeddings_proj, image_labels)
                        # print(recall_score)
                        loss.backward()
                        # print("-------------------------------------------")
                        # for name, param in self.model.named_parameters():
                        #   if param.grad is not None:
                        #       print(f"{name}: grad norm = {param.grad.norm()}")
                        self.optimizer.step()
                        # self.optimizer2.step() #decoder optimizer
                        total_loss_actual += loss.item()


                    else: ##Eval mode
                        ###Test the last batches scores
                        with torch.no_grad():
                            output = self.model(images, image_labels, recipe_enc_src)
                            mmr_logits = output["mmr_logits"]
                            image_logits = output["image_logits"]
                            image_embeddings_proj = output["image_embeddings_proj"]
                            recipe_embeddings_proj = output["recipe_embeddings_proj"]
                            # #Train the Recipe Decoder so it can output a generative (Title, Ingredient, Instructions)
                            # title_logits = self.recipe_decoder(image_embeddings, titles)
                            # ingredients_logits = self.recipe_decoder(image_embeddings, ingredients)
                            # instructions_logits = self.recipe_decoder(image_embeddings, instructions)

                            # #Compute Reconstruction Loss
                            # title_loss = self.CELoss(title_logits.transpose(1, 2), titles)
                            # ingredients_loss = self.CELoss(ingredients_logits.transpose(1, 2), ingredients)
                            # instructions_loss = self.CELoss(instructions_logits.transpose(1, 2), instructions)
                            # decoder_loss = (title_loss + ingredients_loss + instructions_loss) * self.decoder_lambda
                            eval_loss = self.loss_calcs.total_eval_loss(image_logits, image_embeddings_proj,
                                                                   recipe_embeddings_proj, margin=self.fixed_margin)  # + decoder_loss

                            total_eval_loss += eval_loss.item()


                            # recall_score = calc_recall(self.topk, image_embeddings_proj, recipe_embeddings_proj, image_labels)
                            # print(recall_score)
                            # all_image_labels.append(image_labels)
                            # all_recipe_embeddings.append(recipe_embeddings_proj)
                            # all_image_embeddings.append(image_embeddings_proj)
                    # print(f"Batch: {i}/{len(self.dataloader[phase])}")
                    # del unused_tensor
                    torch.cuda.empty_cache() #clear cache after each batch
                    # print(i)
                    # print(output)
                    # total_loss += loss.item()
                # if phase == "eval":
                #     all_image_labels = [item for sublist in all_image_labels for item in sublist]
                #     recipe_embeddings_total = torch.cat(all_recipe_embeddings, dim=0)
                #     image_embeddings_total = torch.cat(all_image_embeddings, dim=0)
                #     # print(f"Image Labels Shape: {len(all_image_labels)}")
                #     # print(f"Recipe Embeddings Shape: {recipe_embeddings_total.shape}")
                #     # print(f"Image Embeddings Shape: {image_embeddings_total.shape}")
                #     recall_score = calc_recall(self.topk, image_embeddings_total, recipe_embeddings_total, all_image_labels)
                #     print(f"Recall Score: {recall_score}")

                # print(f"{phase}: Epoch {epoch+1}, Loss: {total_loss_actual / len(self.dataloader[phase])}")#, with Recall Score: {recall_score}")

                # Update loss margin
                if self.margin <= (self.max_margin - self.margin_step):
                    self.margin += self.margin_step
                else:
                    self.margin = self.max_margin
                if phase == "train":
                  self.train_loss_list.append(total_loss_actual/len(self.dataloader[phase]))
                  print(f"{phase}: Epoch {epoch+1}, Loss: {total_loss_actual / len(self.dataloader[phase])}")#, with Recall Score: {recall_score}")

                else:
                  self.eval_loss_list.append(total_eval_loss/len(self.dataloader[phase]))
                  print(f"{phase}: Epoch {epoch+1}, Loss: {total_eval_loss / len(self.dataloader[phase])}")#, with Recall Score: {recall_score}")
                  total_loss = total_eval_loss/len(self.dataloader[phase])
                  if total_loss < self.best_loss:
                      self.best_loss = total_loss
                      torch.save(self.model.state_dict(), self.best_model_parameters)
                      print("saving this model")
                      ###save the model to then calculate the embedded space



    ##Waiting on training code to finish
    def plot_learning_loss_curves(self):
        """
        Plot accuracy and loss curves for training and eval accuracy/loss lists (item/epoch)
        """
        plt.figure(figsize=(10, 5))
        plt.plot(self.train_loss_list, label='Training Loss')
        plt.plot(self.eval_loss_list, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Loss Curve')
        plt.legend()
        plt.show()

        # plt.figure(figsize=(10, 5))
        # plt.plot(self.train_acc_list, label='Training Accuracy')
        # plt.plot(self.eval_acc_list, label='Validation Accuracy')
        # plt.xlabel('Epoch')
        # plt.ylabel('Accuracy')
        # plt.title('Accuracy Curve')
        # plt.legend()
        # plt.show()


In [ ]:
%autoreload 2

In [ ]:
print(len(filtered_df['tokenized_ingredients'][1][0]))
print(len(filtered_df['tokenized_instructions'][1][0]))
print(len(filtered_df['tokenized_titles'][1][0]))

128
128
128


In [ ]:
# torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
stop

NameError: name 'stop' is not defined

In [ ]:
# Cuda issues
print(torch.__version__)
print(torch.cuda.is_available())
vocab_size = tokenizer_recipes.vocab_size
param_path = os.path.join(GOOGLE_DRIVE_PATH, "models/best_model_jake_12_10.pth")
recipe_embeddings_path  = os.path.join(GOOGLE_DRIVE_PATH, "models/recipe_embeddings.pth")

kwargs = {
    'epochs': 10,
    'ingredient_tokens': filtered_df['tokenized_ingredients'].to_list(),
    'instruction_tokens': filtered_df['tokenized_instructions'].to_list(),
    'title_tokens': filtered_df['tokenized_titles'].to_list(),
    'image_tensors': filtered_tensors,
    'image_labels': filtered_df['Image_Name'],
    'device': device,
    'vocab_size': vocab_size,
    'max_len': total_max,
    'clip_model': clip_model,
    'optimizer': 'adam',
    'learning_rate': 1e-1,
    'batch_size': 15,
    'instance_weight': 1,
    'sem_weight': 5,
    'itm_weight': 0.5,
    'initial_margin': 5.0,
    'margin_step': 1,
    'max_margin': 1000.0,
    'best_model_parameters_path': param_path,
    'decoder_lambda': 0.1,
    'topk': 10,
    'patience': 6,
    'improvement_threshold': 0.001,
    'projection_dim': 1024,
    'fixed_margin': 20,
    'hidden_dim' : 512,
    'mmr_heads': 2,
    'ITEM_lyrs': 1,
    'MTD_lyrs': 1,
    'MMR_type': 'type_separate'

}
image2recipe = Trainer(**kwargs)
image2recipe.train()
image2recipe.plot_learning_loss_curves()

Now that the model is trained, use it to guess images recipes!

In [ ]:
stop

Create the recipe Embedded space with the best parameters (lowest loss)

In [113]:
recipe_embeddings_path = os.path.join(GOOGLE_DRIVE_PATH, "models/recipe_embeddings_proj_chel_11.pth")
image_embeddings_path = os.path.join(GOOGLE_DRIVE_PATH, "models/image_embeddings_proj_chel_11.pth")
param_path = os.path.join(GOOGLE_DRIVE_PATH, "models/recipe_checkpoint_12-09_22-53__Trial_4-best_model.pth")
vocab_size = tokenizer_recipes.vocab_size
from models.MMR import MMR

print(filtered_tensors.shape)
print(filtered_df.shape)
mmr_heads = 4
MMR_type = 'type_seperate'
projection_dim = 512
ITEM_lyrs = 4
MTD_lyrs = 4


print(filtered_df.shape)
##load up the model
def load_model(model_path):
    num_classes = len(set(filtered_df['Image_Name']))
    max_len = total_max
    image_encoder = Image_Encoder(device, clip_model, num_classes).to(device)
    recipe_encoder = RecipeEncoder(device, vocab_size, max_len).to(device)
    print(image_encoder.clip_model.config.projection_dim)
    # mmr = MMR(hidden_dim=image_encoder.clip_model.config.projection_dim).to(device)
    mmr = MMR(hidden_dim=image_encoder.clip_model.config.projection_dim, num_heads=mmr_heads,
              ITEM_lyrs=ITEM_lyrs, MTD_lyrs=MTD_lyrs, projection_dim=projection_dim, MMR_type=MMR_type).to(device)

    # mmr = MMR(hidden_dim=1024).to(device)
    model = Image2Recipe(image_encoder, recipe_encoder, mmr).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

model = load_model(param_path)

data_total = Data_Loading(
            filtered_df['tokenized_ingredients'].to_list(),
            filtered_df['tokenized_instructions'].to_list(),
            filtered_df['tokenized_titles'].to_list(),
            filtered_tensors,
            filtered_df['Image_Name']
        )
batch_size=10
positives_total = 0
dataloader = DataLoader(data_total, batch_size=batch_size, shuffle=False, drop_last=True)
##Create the recipe embedded space
all_recipe_embeddings = []
all_image_embeddings = []
selected = [1,2,3]
for i, batch in enumerate(dataloader):
  with torch.no_grad():
    ingredients, instructions, titles, images, image_labels = (
        batch['ingredients'].to(device),
        batch['instructions'].to(device),
        batch['titles'].to(device),
        batch['images'].to(device),
        batch['image_labels']
    )
    recipe_enc_src = [titles, ingredients, instructions]
    output = model(images, image_labels, recipe_enc_src)
    image_projection = output['image_embeddings_mmr']
    recipe_projection = output["recipe_projection_mmr"]
    image_indices = np.arange(0,batch_size)
    # positives = calc_recall_showimages(10, image_projection, recipe_projection, image_indices, selected, image_labels)
    positives = calc_recall_id(1, image_projection, recipe_projection, image_indices)
    positives_total += positives
    recall_score = positives/batch_size
    print(f"Recall Score: {recall_score}")
    all_image_embeddings.append(image_projection)
    all_recipe_embeddings.append(recipe_projection)
    print(f"Batch: {i}/{len(dataloader)}")

total_recall = (positives_total/(len(dataloader)*batch_size))*100
print(f"Total Recall Score: {total_recall}%")
recipe_embeddings_total = torch.cat(all_recipe_embeddings, dim=0)
image_embeddings_total = torch.cat(all_image_embeddings, dim=0)
# # print(recipe_embeddings_total.shape)
torch.save(recipe_embeddings_total, recipe_embeddings_path)
torch.save(image_embeddings_total, image_embeddings_path)


torch.Size([4968, 3, 224, 224])
(4968, 9)
(4968, 9)
512


<ipython-input-113-3d0d354d072f>:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


total found =  4
Recall Score: 0.4
Batch: 0/496
total found =  1
Recall Score: 0.1
Batch: 1/496
total found =  1
Recall Score: 0.1
Batch: 2/496
total found =  1
Recall Score: 0.1
Batch: 3/496
total found =  1
Recall Score: 0.1
Batch: 4/496
total found =  1
Recall Score: 0.1
Batch: 5/496
total found =  0
Recall Score: 0.0
Batch: 6/496
total found =  1
Recall Score: 0.1
Batch: 7/496
total found =  2
Recall Score: 0.2
Batch: 8/496
total found =  1
Recall Score: 0.1
Batch: 9/496
total found =  1
Recall Score: 0.1
Batch: 10/496
total found =  2
Recall Score: 0.2
Batch: 11/496
total found =  1
Recall Score: 0.1
Batch: 12/496
total found =  1
Recall Score: 0.1
Batch: 13/496
total found =  1
Recall Score: 0.1
Batch: 14/496
total found =  0
Recall Score: 0.0
Batch: 15/496
total found =  2
Recall Score: 0.2
Batch: 16/496
total found =  1
Recall Score: 0.1
Batch: 17/496
total found =  1
Recall Score: 0.1
Batch: 18/496
total found =  1
Recall Score: 0.1
Batch: 19/496
total found =  2
Recall Score:

Test images from dataset to get a recall score

In [114]:
recipe_embeddings_path = os.path.join(GOOGLE_DRIVE_PATH, "models/recipe_embeddings_proj_chel_11.pth")
image_embeddings_path = os.path.join(GOOGLE_DRIVE_PATH, "models/image_embeddings_proj_chel_11.pth")
recipe_embeddings = torch.load(recipe_embeddings_path)
image_embeddings = torch.load(image_embeddings_path)
print(recipe_embeddings.shape)
print(image_embeddings.shape)

image_indices = np.arange(0,image_embeddings.shape[0])
print(image_indices.shape)
topk_vals = [1,5,10]
selected = [10, 35, 600]
for topk in topk_vals:
  recall_score = calc_recall_id(topk, image_embeddings, recipe_embeddings, image_indices)

  print(f"Recall score for {topk}: {round(recall_score/image_embeddings.shape[0] *100,3)}%")

<ipython-input-114-dd6ac7a2ade4>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  recipe_embeddings = torch.load(recipe_embeddings_path)
<ipython-input-114-dd6ac7a2ade4>:4: 

torch.Size([4960, 512])
torch.Size([4960, 512])
(4960,)
total found =  1
Recall score for 1: 0.02%
total found =  5
Recall score for 5: 0.101%
total found =  10
Recall score for 10: 0.202%


In [ ]:
recipe_embeddings_path = os.path.join(GOOGLE_DRIVE_PATH, "models/recipe_embeddings_12_10_jake_noimage.pth")
param_path = os.path.join(GOOGLE_DRIVE_PATH, "models/best_model_jake_12_10.pth")
recipe_embeddings = torch.load(recipe_embeddings_path)
vocab_size = tokenizer_recipes.vocab_size
mmr_heads = 2
MMR_type = 'type_separate'
projection_dim = 1024
ITEM_lyrs = 1
MTD_lyrs = 1

##load up the model
def load_model(model_path):
    num_classes = len(set(filtered_df['Image_Name']))
    max_len = total_max
    image_encoder = Image_Encoder(device, clip_model, num_classes).to(device)
    recipe_encoder = RecipeEncoder(device, vocab_size, max_len).to(device)
    mmr = MMR(hidden_dim=image_encoder.clip_model.config.projection_dim, num_heads=mmr_heads,
              ITEM_lyrs=ITEM_lyrs, MTD_lyrs=MTD_lyrs, projection_dim=projection_dim, MMR_type=MMR_type).to(device)
    model = Image2Recipe(image_encoder, recipe_encoder, mmr).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

model = load_model(param_path)

data_total = Data_Loading(
            filtered_df['tokenized_ingredients'].to_list(),
            filtered_df['tokenized_instructions'].to_list(),
            filtered_df['tokenized_titles'].to_list(),
            filtered_tensors,
            filtered_labels
        )
batch_size=10
dataloader = DataLoader(data_total, batch_size=batch_size, shuffle=False, drop_last=True)
##Create the recipe embedded space
all_recipe_embeddings = []
for i, batch in enumerate(dataloader):
  with torch.no_grad():
    ingredients, instructions, titles, images, image_labels = (
        batch['ingredients'].to(device),
        batch['instructions'].to(device),
        batch['titles'].to(device),
        batch['images'].to(device),
        batch['image_labels']
    )

    recipe_enc_src = [titles, ingredients, instructions]
    output = model(images, image_labels, recipe_enc_src)
    image_embeddings_proj = output["image_embeddings_proj"]
    # plot_2D(recipe_embeddings, image_embeddings_proj, multiplier=i, batch_size = batch_size)
    recall_score = calc_recall(10, image_embeddings_proj, recipe_embeddings, image_labels)
    print(recall_score)
    print(f"Batch: {i}/{len(dataloader)}")
    torch.cuda.empty_cache()


Test the model on 1000 pictures

In [93]:
from PIL import Image
import torchvision.transforms as transforms
recipe_embeddings_path = os.path.join(GOOGLE_DRIVE_PATH, "models/recipe_embeddings_proj_em_11.pth")
recipe_embeddings = torch.load(recipe_embeddings_path)
param_path = os.path.join(GOOGLE_DRIVE_PATH, "models/best_model_jake_12_10.pth")
mmr_heads = 2
MMR_type = 'type_separate'
projection_dim = 1024
ITEM_lyrs = 1
MTD_lyrs = 1
transform = transforms.Compose([
    transforms.Resize((224, 224)),  #same size as training
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #same norm as training
])

def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0) #add batch dim of 1 at 0 indice

def extract_image_features(image, model, device):
  model.eval()
  with torch.no_grad():
      image_logits, image_features = model.image_encoder(image.to(device), image_labels)
  return image_features

def load_model(model_path):
    num_classes = len(set(filtered_df['Image_Name']))
    max_len = total_max
    image_encoder = Image_Encoder(device, clip_model, num_classes).to(device)
    recipe_encoder = RecipeEncoder(device, vocab_size, max_len).to(device)
    mmr = MMR(hidden_dim=image_encoder.clip_model.config.projection_dim, num_heads=mmr_heads,
              ITEM_lyrs=ITEM_lyrs, MTD_lyrs=MTD_lyrs, projection_dim=projection_dim, MMR_type=MMR_type).to(device)
    model = Image2Recipe(image_encoder, recipe_encoder, mmr).to(device)
    model.load_state_dict(torch.load(model_path))
    return model

image_path = os.path.join(GOOGLE_DRIVE_PATH, "Data/test_images/cookie.png")
tensor_image = preprocess_image(image_path)
image2recipe = load_model(param_path)
image_features = extract_image_features(tensor_image, model, device)
print(image_features.shape)

recipe_embeddings = torch.load(recipe_embeddings_path)

recall_score = calc_recall_showimages(10, image_features, recipe_embeddings, filtered_df['Image_Name'].to_list())
print(recall_score)

#Compute cosine similarities





<ipython-input-93-55827c419e49>:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


RuntimeError: Error(s) in loading state_dict for Image2Recipe:
	size mismatch for image_encoder.fc1.weight: copying a param with shape torch.Size([996, 512]) from checkpoint, the shape in current model is torch.Size([4968, 512]).
	size mismatch for image_encoder.fc1.bias: copying a param with shape torch.Size([996]) from checkpoint, the shape in current model is torch.Size([4968]).